#  Submit adsorption energy

In [ ]:
# General imports.
import nglview
import numpy as np
import ipywidgets as ipw
from IPython.display import clear_output
import itertools

from traitlets import dlink

# AiiDA imports.
%aiida
from aiida.orm import Dict as Dict
from aiida.orm import SinglefileData, Node, StructureData

# Workchains.
from aiida.plugins import WorkflowFactory



# AiiDA lab imports.
from aiidalab_widgets_base.utils import string_range_to_list, list_to_string_range
from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, BasicStructureEditor
from aiidalab_widgets_base import StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget

# Custom imports.
from apps.surfaces.widgets.create_xyz_input_files import make_geom_file
from apps.surfaces.widgets.ANALYZE_structure import StructureAnalyzer
from apps.surfaces.widgets.build_slab import BuildSlab
from apps.surfaces.widgets.cp2k2dict import CP2K2DICT
from apps.surfaces.widgets.inputs import InputDetails
from apps.surfaces.widgets.empa_viewer import EmpaStructureViewer
from apps.surfaces.widgets.import_cdxml import CdxmlUpload2GnrWidget

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
Cp2kAdsorptionEnergyWorkChain = WorkflowFactory('nanotech_empa.cp2k.ads_ene')

In [ ]:
# Structure selector.
#build_slab = BuildSlab(title='build slab')
empa_viewer = EmpaStructureViewer()
#dlink((empa_viewer, 'details'), (build_slab, 'details'))
#dlink((empa_viewer, 'structure'), (build_slab, 'molecule'))

structure_selector = StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        StructureBrowserWidget(title="AiiDA database"),
        #StructureUploadWidget(title="Import from computer"),
        #SmilesWidget(title="From SMILES"),
        #CdxmlUpload2GnrWidget(title="CDXML"),
    ],
#    editors = [ #If we enable, check the observe on structure..
#        BasicStructureEditor(title="Edit structure") #, build_slab 
#    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computer_code_dropdown = CodeDropdown(input_plugin='cp2k')
#input_details = InputDetails()

In [ ]:
# all widgets

In [ ]:
#spin guess
fragment1 = ipw.Text(
    value='',
    placeholder='1 2 10..13',
    description='Molecule',
    disabled=False
)

fragment2 = ipw.Text(
    value='',
    placeholder='3 14..500',
    description='Substrate',
    disabled=False
)


#Whole Multiplicity
whole_multiplicity = ipw.IntText(
    value=0,
    description='Multiplicity',
    disabled=False
)
#description
html_whole_mult = ipw.HTML(
                    value="""
                <p style="font-weight:400;">Multiplicity of the whole system.
                </p>
               """
                )

#Multiplicity
multiplicity = ipw.Text(
    value='0 0',
    placeholder='1 0',
    description='Fragment mult.:',
    disabled=False
)
#description
html_mult = ipw.HTML(
                    value="""
                <p style="font-weight:400;">Multiplicity of each fragment.
                </p>
               """
                )

#Multiplicity
charge = ipw.Text(
    value='0 0',
    placeholder='0 0',
    description='',
    disabled=False
)
#description
html_charge = ipw.HTML(
                    value="""
                <p style="font-weight:400;">Charge of each fragment.
                </p>
               """
                )

#UKS
uks = ipw.Checkbox(
    value=False,
    description='UKS',
    disabled=False,
    indent=False
)

#spin guess
spins_up = ipw.Text(
    value='',
    placeholder='1 2 10..13',
    description='Spins U:',
    disabled=False
)

spins_down = ipw.Text(
    value='',
    placeholder='3 4 14..17',
    description='Spins D:',
    disabled=False
)

fixed_atoms = ipw.Text(
    value='',
    placeholder='3 4 14..17',
    description='Fixed atoms:',
    disabled=False
)

#max nodes
max_nodes = ipw.IntText(
    value=48,
    description='Max # compute nodes',
    disabled=False,
    style={"description_width": "initial"}
)
#walltime
walltime = ipw.IntText(
    value=86400,
    description='Walltime seconds',
    disabled=False,
    style={"description_width": "initial"}
)
#protocol
protocol = ipw.Dropdown(     value='standard',
    description='Protocol:',
    disabled=False,       options = [("Standard", "standard"), ("Low accuracy", "low_accuracy"),("Debug","debug")])

In [ ]:
def get_builder_ads():
    
    builder = Cp2kAdsorptionEnergyWorkChain.get_builder()
    builder.code = computer_code_dropdown.selected_code
    builder.structure = structure_selector.structure_node
    builder.charge = List(list=[int(q) for q in charge.value.split(' ')])
    builder.multiplicity = List(list=[int(m) for m in multiplicity.value.split(' ')])
    builder.whole_multiplicity = Int(whole_multiplicity.value)
    builder.fixed_atoms = Str(fixed_atoms.value)

    # spin guess
    ase_geom =  structure_selector.structure
    mag_list = [ 0 for t in ase_geom ]
    if uks.value:
        for i in string_range_to_list(spins_up.value)[0]:
            mag_list[i] = 1
        for i in string_range_to_list(spins_down.value)[0]:
            mag_list[i] = -1 
            
        
    builder.magnetization_per_site = List(list=mag_list)
    builder.fragments = List(list=[fragment1.value,fragment2.value])

    builder.protocol = Str(protocol.value)
    

    builder.walltime_seconds = Int(walltime.value)
    builder.max_nodes = Int(max_nodes.value)    
    
    return builder

In [ ]:
def after_submission(_=None):   
    structure_selector.value = None



    
btn_submit_ads = SubmitButtonWidget(Cp2kAdsorptionEnergyWorkChain, 
                                input_dictionary_function=get_builder_ads
                               )
btn_submit_ads.btn_submit.disabled=True

btn_submit_ads.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit_ads.btn_submit.disabled=False
    #check system
    mol_on_surf = empa_viewer.details['system_type'] == 'SlabXY'  
    mol_on_surf = mol_on_surf and  len(empa_viewer.details['all_molecules'])>0    
    msg = 'Adsorption energy for this system not implemented'
    if mol_on_surf:
        btn_submit_ads.btn_submit.disabled=False  
        msg = ''
    spins_up.value = list_to_string_range(empa_viewer.details['spins_up'])
    spins_down.value = list_to_string_range(empa_viewer.details['spins_down'])
    with output:
        clear_output()
        print(msg)
        if uks.value:
            to_display =  [computer_code_dropdown,ipw.HBox([fragment1,fragment2]),ipw.HBox([html_charge,charge]),
                           fixed_atoms,uks,spins_up,spins_down,ipw.HBox([whole_multiplicity,html_whole_mult]),ipw.HBox([multiplicity,html_mult]),protocol,max_nodes,walltime,btn_submit_ads]
        else:
            to_display =  [computer_code_dropdown,ipw.HBox([fragment1,fragment2]),ipw.HBox([html_charge,charge]),fixed_atoms,
                           uks,protocol,max_nodes,walltime,btn_submit_ads]
        display(ipw.VBox(to_display))


structure_selector.observe(update_all, names='structure')
uks.observe(update_all, names='value')

In [ ]:
display(output)